<a href="https://colab.research.google.com/github/AlisonDavey/tinybird_examples/blob/main/wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create a pandas DataFrame from recent changes to Wikipedia

Based on
https://wikitech.wikimedia.org/wiki/Event_Platform/EventStreams#When_not_to_use_EventStreams

In [1]:
!pip install sseclient
!pip install fsspec
!pip install ndjson

  Created wheel for sseclient: filename=sseclient-0.0.27-py3-none-any.whl size=5584 sha256=3b5257112e633175d628728ab04dc1353516216c08c29f605a339fd19a04a647
  Stored in directory: /root/.cache/pip/wheels/07/67/7e/96edf627ac746de1a5c5cbb8d59ed960f033b8352dc12c545d
Successfully built sseclient
     |████████████████████████████████| 133 kB 23.6 MB/s 


In [2]:
import json
import ndjson
from sseclient import SSEClient as EventSource

import fsspec
import time
from google.colab import files

import pandas as pd

In [3]:
df_wiki=pd.DataFrame()

In [4]:
url = 'https://stream.wikimedia.org/v2/stream/recentchange'
n = 15 # minutes

t_end = time.time() + 60 * n
for event in EventSource(url):
  if time.time() > t_end:
    break
  elif event.event == 'message':
          try:
              change = json.loads(event.data)
          except ValueError:
              pass
          else:
            if change['type']!='log':
              df=pd.DataFrame.from_dict(change)
              df_wiki=df_wiki.append(df[df.index=='domain'])

In [ ]:
#df_wiki.drop(columns=['$schema','length','revision'], inplace=True)
#df_wiki.to_json("wiki.ndjson", orient="records", lines=True, force_ascii=0)
#files.download('wiki.ndjson')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Stream from a pandas DataFrame to Tinybird